# IMDB Reviews Sentiment Analyses

Neste notebook será utilizado os dados do Kaggle (https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews).

1. Importar o Dataset
2. Analisar os Dados
3. Preparar os Dados para Construir o Modelo
4. Criar o Dataset de Teste e Treino
5. Treino do Modelo Utilizando Diferentes Algoritmos
6. Avaliação dos Modelos
7. Seleção do Melhor Modelo
8. Deploy do Modelo para o Watson Machine Learning
9. Avaliação do Modelo Final

In [ ]:
!pip install nltk

In [ ]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import json

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# 1 - Importar o Dataset

In [ ]:
# The code was removed by IBM Watson Studio for sharing.

# 2 - Analisar dos dados

In [ ]:
df = df_data_1
df.head()

In [ ]:
df.describe()

# 3 - Preparar os Dados para Construir o Modelo

Preparação dos dados utilizando as seguintes técnicas de Text Feature Engineering:

    1. Tonkenization         
    2. Remover stopwords
    3. Stemming text
    4. Juntar novamente em uma única frase

In [ ]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()

In [ ]:
def identify_tokens(row):
    source = row[0]
    tokens = word_tokenize(source)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

In [ ]:
def remove_stops(row):
    source_tokenization = row[2]
    stop = [w for w in source_tokenization if not w in stop_words]
    return (stop)

In [ ]:
def stem_porter(row):
    my_list = row[2]
    stemmed_list = [porter_stemmer.stem(word) for word in my_list]
    return (stemmed_list)

In [ ]:
def rejoin_words(row):
    my_list = row[2]
    joined_words = (" ".join(my_list))
    return joined_words

In [ ]:
def pre_processing(df):
    print('Tokenization')
    df['text1'] = df.apply(identify_tokens, axis=1)
    print('Remove stop words')
    df['text1'] = df.apply(remove_stops, axis=1)
    print('Stemming')
    df['text1'] = df.apply(stem_porter, axis=1)
    print('Rejoin words')
    df['tidy_text'] = df.apply(rejoin_words, axis=1)
    print('DONE!')
    
    return df

In [ ]:
df = pre_processing(df)

df['tidy_text'] = df['tidy_text'].str.lower()
df.head()

# 4 - Criar o Dataset de Teste e Treino
NOTE: Dataset de teste (30%) e treino (70%) de forma balanceado (Stratified)

In [ ]:
X = df['tidy_text']
Y = df['sentiment']

print(X.shape)
print(Y.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y)

NOTE: Modelos de Machine Learning ou Deep Learning esperam como entrada "X" valores numéricos. Será utilizado o processo de Text Feature Engineering (TFE) Tf-Idf para transformar os textos em valores numéricos.

In [ ]:
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(2,3), sublinear_tf=True)

X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

print(Y.value_counts().shape)
print(X_train_tf.shape)

In [ ]:
le = preprocessing.LabelEncoder()

Y_train_le = le.fit_transform(list(Y_train))
Y_test_le = le.transform(list(Y_test))

# 5 - Treino do Modelo Utilizando Diferentes Algoritmos

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Binary classifiers
# GradientBoostingClassifier
gradient_boost = GradientBoostingClassifier()
gradient_boost.fit(X_train_tf, Y_train_le)
Y_predict_gradient_boost = gradient_boost.predict(X_test_tf)
print('Gradient Boosting Classifier DONE!')

# SVC
svc_model = SVC(gamma='auto', kernel='sigmoid', C=1.8, probability=True)
svc_model.fit(X_train_tf, Y_train_le)
Y_predict_svm = svc_model.predict(X_test_tf)
print('Support Vector Machine(SVM) DONE!')

# RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=10)
random_forest.fit(X_train_tf, Y_train_le)
Y_predict_random_forest = random_forest.predict(X_test_tf)
print('Random Forest Classifier DONE!')

# KNeighborsClassifier
k_neighbors = KNeighborsClassifier()
k_neighbors.fit(X_train_tf, Y_train_le)
Y_predict_k_neighbors = k_neighbors.predict(X_test_tf)
print('K Nearest Neighbor Classifier DONE!')

# LogisticRegression
logistic_regression = LogisticRegression(solver='lbfgs', penalty='l2', C=1.5)
logistic_regression.fit(X_train_tf, Y_train_le)
Y_predict_logistic_regression = logistic_regression.predict(X_test_tf)
print('Logistic Regression DONE!')

In [ ]:
print('Gradient Boosting Classifier:  ', metrics.accuracy_score(Y_test_le, Y_predict_gradient_boost))
print('Support Vector Machine(SVM):   ', metrics.accuracy_score(Y_test_le, Y_predict_svm))
print('Random Forest Classifier:      ', metrics.accuracy_score(Y_test_le, Y_predict_random_forest))
print('K Nearest Neighbor Classifier: ', metrics.accuracy_score(Y_test_le, Y_predict_k_neighbors))
print('Logistic Regression:           ', metrics.accuracy_score(Y_test_le, Y_predict_logistic_regression))

# 6 - Avaliação dos Modelos

## 6.1 - Resumo de classificação
NOTE: Accuracy >= 0.70

In [ ]:
print('Gradient Boosting Classifier:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_gradient_boost)))
print('Support vector machine(SVM):\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_svm)))
print('Random Forest Classifier:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_random_forest)))
print('Logistic Regression:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_logistic_regression)))

## 6.3 - Logistic Regression
NOTE: Matriz de confusão do melhor modelo

In [ ]:
logistic_regression_conf_matrix = metrics.confusion_matrix(Y_test_le, Y_predict_logistic_regression)
sns.heatmap(random_forest_conf_matrix, annot=True,  fmt='');
title = 'Logistic Regression'
plt.title(title);

# 7 - Seleção do Melhor Modelo

In [ ]:
X_train_final = tfidf.fit_transform(X)
Y_train_final = le.fit_transform(list(Y))

print(X_train_final.shape)

In [ ]:
lrc = LogisticRegression(solver='lbfgs', penalty='l2', C=1.5)
lrc.fit(X_train_final, Y_train_final)

# 8 - Deploy do Modelo para o Watson Machine Learning



Para autenticar no Watson Machine Learning no IBM Cloud, você precisa da api_key e location do seu serviço.

Podemos utilizar o [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) ou diretamente pelo portal do IBM Cloud.

Usando o IBM Cloud CLI:

```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

NOTE: Você pode obter a URL do serviço indo até [Endpoint URLs section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).

In [ ]:
api_key = 'API_KEY'
location = 'LOCATION'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": location
}

## 8.1 - Instalando a biblioteca do Watson Machine Learning

NOTE: Documentação pode ser encontrada [aqui](http://ibm-wml-api-pyclient.mybluemix.net/)

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)
print(client.version)

## 8.2 - Criando nosso espaço de implementação

Crie um espaço de implementação pela UI do Watson Studio que será usado para fazer o deploy do nosso modelo.

    1. Clique em "Novo Espaço de Implementação"
    2. Selecione Cloud Object Storage
    3. Selecione Watson Machine Learning e clique em "Criar"
    4. Copie "space_id" e cole abaixo

In [ ]:
space_id = 'SPACE_ID'

In [ ]:
client.spaces.list(limit=10)

In [ ]:
client.set.default_space(space_id)

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
metadata = {
            client.repository.ModelMetaNames.NAME: 'Logistic Regression model to predict IMDB reviews',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=lrc,
    meta_props=metadata)

In [ ]:
published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
client.repository.list_models()

In [ ]:
# client.repository.delete('ID of stored model')

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of IMDB reviews",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

In [ ]:
# Get deployment UID and show details on the deployment
deployment_uid = client.deployments.get_uid(created_deployment)
client.deployments.get_details(deployment_uid)

In [ ]:
client.deployments.list()

In [ ]:
# client.deployments.delete('GUID of deployed model')

# 9 - Avaliação do Modelo Final

NOTE: Testando a nossa API criada com o WML.

In [ ]:
# get scoring end point
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

In [ ]:
# add some test data
scoring_payload = {"input_data": [
    {'values': X_test_tf.toarray()
    }]}

In [ ]:
# score the model
predictions = client.deployments.score(deployment_uid, scoring_payload)
print('prediction',json.dumps(predictions, indent=2))

In [ ]:
Y_predict_final_model = []
for y in predictions['predictions'][0]['values']:
    Y_predict_final_model.append(y[0])
    
print('Final Model WML:\n {}\n'.format(metrics.classification_report(Y_test_le, Y_predict_final_model)))